# Integer Programming Homework

## Total: 40 Points

This notebook guides you through solving various Integer Programming (IP) problems and implementations. 
Follow the instructions provided in each section, write the required code, and answer the questions.

<em>NOTE: You should have the necessary setup from the LP Homework assignment.

# Question 1: Maya's Midnight March (10 points)
Maya's SCOPE team realized that they need someone to go out at midnight wearing white to test the visibility of pedestrians on crosswalks. Maya, who loves her hikes, happily volunteered. She needs to pack light in order to hit all the crosswalks in Needham. 

The maximum weight she wants to carry is 15 lbs. Here are the different items she can bring and their importance for her SCOPE team's data collection: 

+ Reflective Vest: Weight = 2 lbs, Importance = 6
+ Light Meter: Weight = 4 lbs, Importance = 12
+ DSLR Camera: Weight = 10 lbs, Importance = 4 (They really don't want to take pictures)
+ Notebook: Weight = 5 lbs, Importance = 8
+ Police Baton: Weight = 9 lbs, Importance = 10

### Formulate the IP
First, start by setting up the IP similar to the other problems. Write the objective function to maximize the total value of items brought.

Write the equations using LaTeX, between the $$ tags. Then convert the IP problem into code using this documentation: [PuLP Guide](https://realpython.com/linear-programming-python/#using-pulp). Using a binary variable ensures that only integer solutions are provided.

#### Solution:
$$ \text{integer } n \text{ for number of items} \\
\text{binary } y_i \text{ for if item } i \text{ was chosen} \\
\text{integer } w_i \text{ for weight of item } i \\
\text{integer } v_i \text{ for value/importance of item } i \\
\text{integer } W = 15 \text{ for maximum total weight} $$
Objective Function: $$ Z = \sum_{i = 0}^{n} y_iv_i $$ 
Constraint: $$ \sum_{i = 0}^{n} y_iw_i \le W $$ 

In [ ]:
from pulp import *

# Define the Problem
model = LpProblem(name="mayas_march", sense=LpMaximize) # type: ignore

# Define the Variables
# Define each of the decision variables here (each item). Remember to assign them as binary
n = 5
y = [LpVariable(name=f'y{i+1}', cat="Binary") for i in range(n)]
w = [2, 4, 10, 5, 9]
v = [6, 12, 4, 8, 10]
W = 15

# Objective Function
# Add objective function
model += sum([y[i] * v[i] for i in range(n)])

# Constraints
# Add the weight constraint
model += (sum([y[i] * w[i] for i in range(n)]) <= W, "maximum_weight")

# Solve the Model
model.solve()

1

In [4]:
# Print the solution
for var in model.variables():
    print(var.name, var.value())

y1 1.0
y2 1.0
y3 0.0
y4 0.0
y5 1.0


Solution: <em>Write the items selected below.</em>

- Items selected: <b> Reflective Vest, Light Meter, Police Baton</b>

# Question 2: Branch And Bound (30 points)
Branch and bound is one of the most common ways to solve optimization problems, including IP problems. It's a search algorithm that iteratively explores the space to find the optimal solution. 

Start by reading through this explanation: https://web.tecnico.ulisboa.pt/mcasquilho/compute/_linpro/TaylorB_module_c.pdf and writing pseudocode for the algorithm below. Then, we'll work through a scaffolded recursive implementation of the algorithm in Python. 

### Pseudocode

- Find the solution with relaxed integer constraints.
- Your solution is the upper bound, and the rounded down solution becomes the lower bound.
- Constrain the variable with the greatest fractional part by making a $\le$ and a $\ge$ constraint for the adjacent integers, then branch.
- Solve each new node, the solution becomes the upper bound for each node, and the best integer solution is the global lower bound.
- If you have a velid integer solution, there is no feasible solution, or the upper bound of a node is lower than the global lower bound, it's an end node. 
- If you've gotten to all of the end nodes, your best integer solution is the optimal soution with the integer constraints.

### Implementation

We start by defining our arrays for the system (c, A, b). c is an array of the coefficients of the objective function, and A and b are for each constraint. For example: 

$$ Z = 4x_1 + 3 x_2 + x_3$$
$$ 3x_1 + 2x_2 + x_3 <= 7$$
$$ 2x_1 + x_2 + 2x_3 <= 11$$

From here, c = [4, 3, 1], A is [[3, 2, 1], [2, 1, 2]], and b is [7, 11]. 

If there are any upper or lower bounds on any variables, add those to the lb and ub list.

In [12]:
# Similar to the text, we'll have a function to find the relaxed solution of the IP (including decimal points)
import scipy
import numpy as np
from copy import deepcopy
import math

"""
Python implementation of branch and bound algorithm for integer programming problems.

Args:
    c: array with objective function coefficients 
    A: 2D array with constraint coefficients
    b: array with constraint values
    lb: array with lower bound values for each variable
    ub: array with upper bound values for each variable

Returns: an array for values of each variable and the optimal objective function value
"""
def branch_and_bound(c, A, b, lb, ub):
    optimized = scipy.optimize.linprog(-c, A, b, method="highs", bounds=list(zip(lb, ub))) # Function to optimize current 

    x = optimized.x
    z = optimized.fun

    # First check if the problem is feasible or not (check if Z exists).
    if z is None:
        return None, None
    
    # Then, check if the current variables are all integers or not. 
    all_integers = None
    if all([n.is_integer() for n in x]):
        all_integers = True
    else:
        all_integers = False
    
    # If all variables are integers, we have reached the solution 
    if all_integers:
        return x, z
    
    # Otherwise we need to branch the value with the most decimal difference similar to how the paper explained
    else:
        # Start by copying the lower bounds and upper bounds into new arrays. This is how we establish branching
        left_lb = deepcopy(lb)
        left_up = deepcopy(ub)

        right_lb = deepcopy(lb)
        right_up = deepcopy(ub)

        # Find which variable has the maximum decimal difference and save the index (HINT: what math operation deals with remainders?)
        max_difference_idx = np.argmax(np.mod(x, 1))

        # Set the upper and lower bounds based on branching at that variable (floor and ceiling)
        left_up[max_difference_idx] = math.floor(x[max_difference_idx])
        right_lb[max_difference_idx] = math.ceil(x[max_difference_idx])

        # Recursively get values for the optimized solution of the left and right nodes
        x_left, z_left = branch_and_bound(c, A, b, left_lb, left_up)
        x_right, z_right = branch_and_bound(c, A, b, right_lb, right_up)

        # Return the coefficients with the higher value of the two (HINT: This takes an if statement)
        # If the left one was infeasible
        if z_left is None:
            # and the right one was infeasible, return None
            if x_right is None:
                return None, None
            # otherwise, the right one worked, return it
            return x_right, z_right
        
        # If the right one was infeasible, the left one exists, return it
        if x_right is None:
            return x_left, z_left

        # IF they both exist, return the greater one.
        if z_left > z_right:
            return x_left, z_left
        else: 
            return x_right, z_right

Now, lets test the function you've written on the sample problem written above. 

In [13]:
c = np.array([4, 3, 1])
A = np.array([[3, 2, 1], [2, 1, 2]])
b = np.array([7, 11])

lb = np.array([0, 0, 0])
ub = np.array([None, None, None])

x, z = branch_and_bound(c, A, b, lb, ub)

print("Variable")
for idx, var in enumerate(x): 
    print(f">  x{idx} = {var}")

print(f"Objective Function: {z}")

Variable
>  x0 = 1.0
>  x1 = 2.0
>  x2 = 0.0
Objective Function: -10.0


If correct, your solution should be x0 = 1, x1 = 2, and x2 = 0